In [1]:
# Script to load raw translation strings from Excel, then generate a TamperMonkey script
# to do find-and-replace against the AWS DeepRacer console
import openpyxl

wb = openpyxl.load_workbook(filename = 'translations.xlsx')
ws = wb.active

In [2]:
# Tampermonkey script (head)
head = '''// ==UserScript==
// @name         SimplifiedChineseDeepRacer
// @namespace    SimplifiedChineseDeepRacer
// @version      1.0.2
// @description  Translates DeepRacer console into Simplified Chinese
// @author       jdaped
// @match        https://*.console.aws.amazon.com/deepracer/*
// @updateURL    https://jeremypedersen.com/resources/tampermonkey/SimplifiedChineseDeepRacer.user.js
// @downloadURL  https://jeremypedersen.com/resources/tampermonkey/SimplifiedChineseDeepRacer.user.js
// ==/UserScript==

// This timeout is a "hack" to wait for the page to load
// TODO: Find a more elegant way to wait for page load
setTimeout(function () {

    var replaceArray = ['''

In [3]:
# Tampermonkey script (tail)
tail = '''    ];

    var numTerms = replaceArray.length;
    var txtWalker = document.createTreeWalker(
        document.body,
        NodeFilter.SHOW_TEXT,
        {
            acceptNode: function (node) {
                //-- Skip whitespace-only nodes
                if (node.nodeValue.trim()) {
                    return NodeFilter.FILTER_ACCEPT;
                }

                return NodeFilter.FILTER_SKIP;
            }
        },
        false
    );
    var txtNode = null;

    while (txtNode = txtWalker.nextNode()) {
        var oldTxt = txtNode.nodeValue;

        for (var J = 0; J < numTerms; J++) {
            oldTxt = oldTxt.replace(replaceArray[J][0], replaceArray[J][1]);
        }
        txtNode.nodeValue = oldTxt;
    }

}, 3000);

'''

In [4]:
# Sort and convert into a format Tampermonkey can use
# We assume the following format
# Col A   | Col B   | Col C
# English | Chinese | Console URL

# Formatted output array
# (Assume English is in Column B, Chinese in C)
output = []

for i in range(2, ws.max_row + 1):
    try:
        english = ws['B' + str(i)].value.strip()    
    except:
        continue # Ignore empty cells
        
    try:
        chinese = ws['C' + str(i)].value.strip()
    except: 
        continue # Ignore empty cells
    
    # We add the length of the English content to use as a sort key later
    output.append("[/{}/gi, '{}']".format(english, chinese))

# Sort all the terms in the output array by length (longest to shortest)
output.sort(key=lambda x: len(x), reverse=True)

# Open up a file to write our Tampermonkey script to


# Open up a file and dump the raw array content in
f = open('SimplifiedChineseDeepRacer.user.js', 'w')

# Write in header
f.write(head + '\n')

# Write array content
dupes = []
array_len=len(output)
for i in range(0,array_len):
    
    # (and while we are at it, check for + report duplicates)
    if output[i] in dupes:
        print('Duplicate string: {}'.format(output[i]))
    else:
        dupes.append(output[i])
    
    if i != array_len-1:
        f.write('\t'*2 + output[i] + ',\n')
    else:
        f.write('\t'*2 + output[i]) 
        
# Write footer and close file
f.write('\n' + tail + '\n')
f.close()

print("Done!")

Done!
